In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns 


#в конце надо удалить
from sklearn import metrics

In [448]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [449]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
Id                     10000 non-null int64
age                    10000 non-null float64
years_of_experience    10000 non-null float64
lesson_price           10000 non-null float64
qualification          10000 non-null float64
physics                10000 non-null float64
chemistry              10000 non-null float64
biology                10000 non-null float64
english                10000 non-null float64
geography              10000 non-null float64
history                10000 non-null float64
mean_exam_points       10000 non-null float64
dtypes: float64(11), int64(1)
memory usage: 937.6 KB


In [457]:
X = df_train[['years_of_experience','lesson_price','qualification','physics']]
y = df_train['mean_exam_points']
train_data, test_data, train_labels, test_labels = train_test_split(X, y, test_size = 0.01, random_state = 1)

train_data.reset_index(inplace=True, drop=True)
test_data.reset_index(inplace=True, drop=True)
train_labels.reset_index(inplace=True, drop=True)
test_labels.reset_index(inplace=True, drop=True)
X_final = df_test[['years_of_experience','lesson_price','qualification','physics']]

In [451]:
# """
# Node
#     => инициализируемся и находим место разделения
#         =>Создаём левую ветку
#             Node Инициализируемся в левой веткой
#         =>Создаём правую ветку
#             Node Инициализируемся в правой веткой
#         Цикл замкнулся пока всё не кончится листьями или глубиной
    
# """
class Node:
    
    def __init__(self, X, y, indexes, min_samples_leaf=5, min_samples_split=2, max_depth=10):
        """ Инициализация узла. 
        При создании узла сюда передаются следующие параметры:
            X - массив с исходными данными
            y - массив с результатами
            indexes - массив с индексами записей массивов X и y. Если использоваться indexes, то можно будет снизить
                ресурсоёмкость процесса, т.к. все операции можно будет производить лишь гоняя туда-обратно индексы,
                а не целые, неустанно плодящиеся массивы. По крайней мере это должно позволить сэкономить оперативную память
            min_leaf - минимальное количество элементов в листе

            len(X)==len(y)==len(indexes)
        """
        # На основе полученных данных, мы заполняем переменные self. которые будут "глобальными" внутри этого класса.
        self.X = X
        self.y = y
        self.indexes = np.array(indexes) #Преобразуем список индексов в np.array если вдруг его не преобразовывали ранее
        self.split_index = None #
        self.split_value = None
        self.min_samples_leaf = min_samples_leaf
        self.min_samples_split = min_samples_split
        self.rows = len(indexes) #Количество строк в массиве X
        self.cols = X.shape[1] #Количество столбцов в массиве X
        self.mean = np.mean(y[self.indexes]) #Среднее арифметическое всех элементов массива y, чьи индексы есть в массиве indexes. Это и есть прогноз для листа
        self.quality = float('inf') #для того чтобы следить за улучшением модели, задаём предельное значение с которым будем
        #сравнивать первый проход
#         self.my_get_mse_score(np.array([2,4,4,4,5,5,7,9]),np.array([0]))
        self.make_tree()
        
    
    def make_tree(self):
        """ Функция постройки дерева
        1) На данный момент, всё что мы имеем - это инициализированная нода и в конце инициализации была вызвана эта функция
        2) Нужно 
        """
        for col in range(self.cols): #Проходим слева на право по колонкам
            #отправляя номер каждой колонки массива X в функцию поиска наилучшего разбиения
            self.find_best_split(col)
#         print(self.quality, self.split_index, self.split_value)   
#         print(self.X.values[self.indexes,self.split_index])
#         print(len(self.X.values[self.indexes,self.split_index]))
        
        
#         raise
        """ Если find_best_split, ни разу не дошла до обновления self.quality и всегда отсеивалась при проверках
        на всякие ограничения, значит это лист
        """
        if self.quality == float('inf'):
                return #А значит мы дерево дошло до одной из веток и мы прекращаем её построение
            
        #В противном случае
        #В этот момент мы имеем 3 параметра наилучшего разделения: self.quality, self.split_index, self.split_value
        # По ним нам нужно совершить окончательное разделение данных на 2 ветки и с каждой из них инициализировать ноду
#             left = np.array(X<=)
        #записываем в X текущие значения по которым будем разбивать
        X = self.X.values[self.indexes,self.split_index]
        left = np.nonzero(X <= self.split_value)[0] #отделяем индексы наблюдений что пойдут влево
        right = np.nonzero(X > self.split_value)[0] #отделяем индексы наблюдений что пойдут вправо
#         print(f"left={left}")
#         print(f"self.indexes[left]={self.indexes[left]}")
        self.left = Node(self.X, self.y, self.indexes[left], self.min_samples_leaf, self.min_samples_split)
        self.right = Node(self.X, self.y, self.indexes[right], self.min_samples_leaf, self.min_samples_split)
#         print(x)
        
    def find_best_split(self, col):
        """ Функция поиска наилучшего разбиения узла на две ветки
        1) Мы находимся в текущей ноде
        2) В момент инициализации в эту ноду были переданы ссылки на массив X и y, а так же массив indexes, содержащий
            индексы элементов, относящихся именно к этой ноде
        3) Сюда передаётся только номер текущей колонки, по которой мы будем искать наилучшее разбиение 
        """
        #Таки начинаем перебирать
        """ Выбираем из массива X, все записи с индексами указанными в массиве self.indexes 
            и берём из них текущую колонку
        """ 
#         print(self.X.values[self.indexes, col])
#         raise
        X = self.X.values[self.indexes, col]
#         print(X)
        #В результате, мы берём из всего датафрейма, для всех наблюдений, конкретную колонку в одномерный массив
        # Например возраст. Теперь в X у нас параметр Возраст из всех наблюдений датафрейма

        #Теперь, когда у нас есть одномерный массив, пробегаем по нему слева направо, на каждом этапе разделяя
        #его на левую и правую части
        for cur_param in range(self.rows):
#             print(cur_param)
            
#             print(f"cur_param:{cur_param}")
#             print(f"X:{X}")
#             print(f"X[cur_param]:{X[cur_param]}")
#             print(f"len(X):{len(X)}")
#             print(f"self.rows={self.rows}")
            left = np.where(X <= X[cur_param]) #В левую часть отправляем индексы всего что <= текущему значению в X[cur_param]
            right = np.where(X > X[cur_param]) #В правую часть отправляем индексы всего что > текущего значения в X[cur_param]
            
            #После разбиения на два, нам нужно отделить X ушедшие в право, от ушедших влево, аналогично с y
            true_data = self.X.values[left]
            false_data = self.X.values[right]
            true_labels = self.y.values[left]
            false_labels = self.y.values[right]
            
            if len(true_data) < self.min_samples_leaf or len(false_data) < self.min_samples_leaf:
#                 print(f"Ограничение в {self.min_samples_leaf} элементов в листе")
                continue
            
            score = self.my_get_mse_score(true_labels, false_labels)
            if score < self.quality:
                self.quality = score
                self.split_index = col
                self.split_value = X[cur_param]
#                 print(self.quality, self.split_index, self.split_value)
        
    def my_get_mse_score(self, left, right):
        len_l = len(left)
        len_r = len(right)
#         print(f"left:{left}")
#         print(f"right:{right}")
#         print(len_l,len_r)
        len_m = len_l + len_r
        mse_left = (sum((left - left.mean())**2)/len(left))**1/2
        mse_right = (sum((right - right.mean())**2)/len(right))**1/2
        return (len_l/len_m)*mse_left + (len_r/len_m)*mse_right
        
    def predict(self, X):
        """Функция подачи данных массива в функцию предсказания
        Принимает на вход массив X, содержащий параметры, идентичные тому массиву на котором обучалось дерево
        Разбивает масив на отдельные строки и передаёт их непосредственно в функцию предсказания.
        Возвращает результирующий массив y_pred содержащий предсказанные значения
        """
        return np.array([self.predict_row(X_row) for X_row in X])
            
#         return np.array([self.predict_row(xi) for xi in x])
        
    def predict_row(self, X_row):
#         print(f"type(X_row)={type(X_row)}")
#         print(f"X_row={X_row}")
#         print(f"type(self.split_value)={type(self.split_value)}")
#         print(f"self.split_value={self.split_value}")
        if self.quality == float('inf'):
            #Если текущий узел является листом - возвращаем его среднее значения
            return self.mean
        #Иначе мы обрабатываем X_row
        if X_row[self.split_index] <= self.split_value:
            node = self.left
        else:
            node = self.right
        return node.predict_row(X_row)
#     n_estimators=10, 
# max_depth = None
    

In [452]:
test = Node(train_data, train_labels, train_data.index.tolist())

In [453]:
pred_train = test.predict(train_data.values)

In [454]:
metrics.r2_score(train_labels, pred_train)

0.7876794649254705

In [455]:
pred_test = test.predict(test_data.values)

In [456]:
metrics.r2_score(test_labels, pred_test)

0.807066341386015

In [458]:
answers = test.predict(X_final.values)

In [460]:
answers

array([52.15      , 60.53846154, 49.16      , ..., 53.3       ,
       65.45783133, 72.625     ])

In [461]:
X_final

,years_of_experience,lesson_price,qualification,physics
0,3.0,1050.0,1.0,0.0
1,3.0,1850.0,2.0,0.0
2,1.0,1550.0,1.0,1.0
3,6.0,2900.0,3.0,1.0
4,4.0,3150.0,3.0,1.0
5,0.0,1350.0,1.0,0.0
6,0.0,1500.0,1.0,0.0
7,0.0,1150.0,1.0,0.0
8,2.0,1550.0,2.0,0.0
9,0.0,2250.0,3.0,0.0


In [462]:
df_test

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history
0,10000,46.0,3.0,1050.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,10001,43.0,3.0,1850.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10002,52.0,1.0,1550.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,10003,57.0,6.0,2900.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0
4,10004,44.0,4.0,3150.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0
5,10005,51.0,0.0,1350.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,10006,38.0,0.0,1500.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
7,10007,50.0,0.0,1150.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
8,10008,51.0,2.0,1550.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10009,58.0,0.0,2250.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
>>> sLength = len(df1['a'])
>>> df1
          a         b         c         d
6 -0.269221 -0.026476  0.997517  1.294385
8  0.917438  0.847941  0.034235 -0.448948

>>> df1['e'] = pd.Series(np.random.randn(sLength), index=df1.index)
>>> df1
          a         b         c         d         e
6 -0.269221 -0.026476  0.997517  1.294385  1.757167
8  0.917438  0.847941  0.034235 -0.448948  2.228131

>>> p.version.short_version
'0.16.1'


df1 = df1.assign(e=pd.Series(np.random.randn(sLength)).values)

In [466]:
df_test = df_test.assign(mean_exam_points=pd.Series(answers))
# df_test.loc[:,'mean_exam_points'] = pd.Series(answers, index=df_test.index)

In [475]:
submissions = df_test.drop(['age','years_of_experience','lesson_price','qualification','physics','chemistry','biology','english','geography','history'], axis=1)

In [476]:
submissions

,Id,mean_exam_points
0,10000,52.150000
1,10001,60.538462
2,10002,49.160000
3,10003,86.500000
4,10004,88.571429
5,10005,63.406250
6,10006,40.204301
7,10007,54.573171
8,10008,59.315789
9,10009,79.876190


In [477]:
submissions.to_csv('submissions.csv', index=False)